In [98]:
import os,re,csv,string,syllables,math,nltk,nltk.data,io,markovify,sys,getopt,os.path,random,json,gensim,requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from nltk.collocations import *
from nltk.tokenize import word_tokenize,RegexpTokenizer
from nltk.util import ngrams
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer,WordNetLemmatizer
from gensim.models import Word2Vec
from random_word import RandomWords
from essential_generators import DocumentGenerator

In [2]:
# 計算音節數量
def count_syllables(word):
    return len(re.findall('(?!e$)[aeiouy]+', word, re.I) + re.findall('^[^aeiouy]*e$', word, re.I))
def syllable_count(word):
    word = word.lower()
    punc = string.punctuation
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    if word in punc: # 如果詞語是標點符號，音節數量則為0
        count = 0
    return count

In [77]:
# 透過爬蟲方式找相似詞語
def crawl_synonyms(word):
    url = "https://www.thesaurus.com/browse/"
    url += word
    #print(url)
    r = requests.get(url)
    soup = bs(r.content, "html.parser")
    answer_list = soup.find("ul",'css-17d6qyx-WordGridLayoutBox et6tpn80')
    syns = answer_list.findChildren("a")
    #print(syns)
    lst = []
    for i in range(len(syns)):
        lst.append(syns[i].text)
    return lst

In [3]:
# 生成歌詞
# 句子詞語分割
# https://www.geeksforgeeks.org/nlp-how-tokenizing-text-sentence-words-works/
# https://stackoverflow.com/questions/15547409/how-to-get-rid-of-punctuation-using-nltk-tokenizer
gen_text = "Where the old men pray\
            His children lived next door\
            They used to live in the woods\
            But this man's religious bone\
            He didn't have a prayer\
            No, he didn't have a prayer\
            He just said I've got to find my own way home\
            And I've got to find my own way home\
            If I can do the things he says\
            That I just can't understand\
            Then I'll just settle down and start living for today\
            Lord, how I'd love to settle down\
            And start living for today\
            Lord, how I'd love to settle down\
            But I don't have a prayer\
            I don't have a prayer\
            No, I don't have a prayer\
            Sometimes, I think of what a wife must be\
            And how I hate to do things\
            That he doesn't have to do\
            But then somehow, I think of what hope is\
            And do I have to start\
            Oh, oh, I'm ready to begin\
            Oh, how I'd love to settle down\
            I'm ready, ready, ready\
            If I can do the things he says\
            That I just can't understand\
            Then I'll just settle down and start living for today\
            Lord, how I'd love to settle down\
            And I don't have a prayer\
            No, I don't have a prayer\
            Always, always start the day\
            Lord, how I'd love to settle down\
            I'm ready, ready, ready\
            If I can do the things he says\
            That I just can't understand\
            Then I'll just settle down and start living for today\
            Lord, how I'd love to settle down\
            But I don't have a prayer No, I don't have a prayer\
            I can't do the things he says\
            That I just can't understand\
            Then I'll just settle down and start living for today"
tokenizer = RegexpTokenizer("[\w']+") 
lst_gen_tokenize = tokenizer.tokenize(gen_text)
print(len(lst_gen_tokenize))
print(lst_gen_tokenize)

288
['Where', 'the', 'old', 'men', 'pray', 'His', 'children', 'lived', 'next', 'door', 'They', 'used', 'to', 'live', 'in', 'the', 'woods', 'But', 'this', "man's", 'religious', 'bone', 'He', "didn't", 'have', 'a', 'prayer', 'No', 'he', "didn't", 'have', 'a', 'prayer', 'He', 'just', 'said', "I've", 'got', 'to', 'find', 'my', 'own', 'way', 'home', 'And', "I've", 'got', 'to', 'find', 'my', 'own', 'way', 'home', 'If', 'I', 'can', 'do', 'the', 'things', 'he', 'says', 'That', 'I', 'just', "can't", 'understand', 'Then', "I'll", 'just', 'settle', 'down', 'and', 'start', 'living', 'for', 'today', 'Lord', 'how', "I'd", 'love', 'to', 'settle', 'down', 'And', 'start', 'living', 'for', 'today', 'Lord', 'how', "I'd", 'love', 'to', 'settle', 'down', 'But', 'I', "don't", 'have', 'a', 'prayer', 'I', "don't", 'have', 'a', 'prayer', 'No', 'I', "don't", 'have', 'a', 'prayer', 'Sometimes', 'I', 'think', 'of', 'what', 'a', 'wife', 'must', 'be', 'And', 'how', 'I', 'hate', 'to', 'do', 'things', 'That', 'he', "

In [4]:
# 計算生成歌詞的音節數量
lst_gen_syllables = []
count1 = []
count2 = []
count3 = []
for i in range(len(lst_gen_tokenize)):
    count_gen = syllable_count(lst_gen_tokenize[i])
    lst_gen_syllables.append(count_gen)
    if count_gen == 1:
        count1.append(lst_gen_tokenize[i])
    if count_gen == 2:
        count2.append(lst_gen_tokenize[i])
    if count_gen == 3:
        count3.append(lst_gen_tokenize[i])
print('lst_gen_syllables:',len(lst_gen_syllables))
print(lst_gen_syllables)

lst_gen_syllables: 288
[1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2]


In [5]:
print(count1)
print(count2)
print(count3)

['Where', 'the', 'old', 'men', 'pray', 'His', 'next', 'door', 'They', 'to', 'live', 'in', 'the', 'woods', 'But', 'this', "man's", 'bone', 'He', "didn't", 'have', 'a', 'prayer', 'No', 'he', "didn't", 'have', 'a', 'prayer', 'He', 'just', 'said', "I've", 'got', 'to', 'find', 'my', 'own', 'way', 'home', 'And', "I've", 'got', 'to', 'find', 'my', 'own', 'way', 'home', 'If', 'I', 'can', 'do', 'the', 'things', 'he', 'says', 'That', 'I', 'just', "can't", 'Then', "I'll", 'just', 'settle', 'down', 'and', 'start', 'for', 'Lord', 'how', "I'd", 'love', 'to', 'settle', 'down', 'And', 'start', 'for', 'Lord', 'how', "I'd", 'love', 'to', 'settle', 'down', 'But', 'I', "don't", 'have', 'a', 'prayer', 'I', "don't", 'have', 'a', 'prayer', 'No', 'I', "don't", 'have', 'a', 'prayer', 'I', 'think', 'of', 'what', 'a', 'wife', 'must', 'be', 'And', 'how', 'I', 'hate', 'to', 'do', 'things', 'That', 'he', "doesn't", 'have', 'to', 'do', 'But', 'then', 'I', 'think', 'of', 'what', 'hope', 'is', 'And', 'do', 'I', 'have'

In [6]:
# 原始歌詞
ori_text = "I'll let you show me his moves\
            Let you do what he taught you\
            Let you reminisce how you used, to do\
            Girl I?m open to anything that'll get you into that zone\
            And understand that we?re all alone\
            So you can slowly take off your clothes\
            Baby girl you know what's in store\
            And baby I will stay up all night\
            I've been goin' hard since last night\
            And I'mma go harder tonight\
            Wish you could see you through my eyes\
            Ooh I'm telling you this ain't the same\
            And I know he's still in your brain\
            I'm 'bout to burn that shit into flames\
            Once I'm in you\
            Baby\
            Forget what you know\
            Make yourself at home\
            'Cuz baby when I'm finished with ya you won't wanna go Outside\
            And I'mma work you like a pro, baby\
            And, you gon' take it like one\
            Yeah you gon' take it like one\
            And I'mma give it like you asked for it\
            Why, cuz you been talkin' 'bout it\
            I know you been talkin' 'bout it\
            Ooh baby when I'm done with you\
            You ain't saying nothing\
            Yeah you ain't saying nothing\
            Ooh baby when I'm done with you\
            You ain't saying nothing\
            Yeah you ain't saying nothing\
            You gon' make me show off\
            All the pain that you feel you can tell that we ain't making no love\
            But I'll pretend\
            Oh girl, I'll pretend\
            If you pretend then girl I'll pretend\
            Let's make it seem like we're all we need in the end\
            Forget what you know\
            Make yourself at home\
            'Cuz baby when I'm finished with ya you won't wanna go Outside"
tokenizer = RegexpTokenizer("[\w']+") 
lst_ori_tokenize = tokenizer.tokenize(ori_text)
print(len(lst_ori_tokenize))
print(lst_ori_tokenize)

277
["I'll", 'let', 'you', 'show', 'me', 'his', 'moves', 'Let', 'you', 'do', 'what', 'he', 'taught', 'you', 'Let', 'you', 'reminisce', 'how', 'you', 'used', 'to', 'do', 'Girl', 'I', 'm', 'open', 'to', 'anything', "that'll", 'get', 'you', 'into', 'that', 'zone', 'And', 'understand', 'that', 'we', 're', 'all', 'alone', 'So', 'you', 'can', 'slowly', 'take', 'off', 'your', 'clothes', 'Baby', 'girl', 'you', 'know', "what's", 'in', 'store', 'And', 'baby', 'I', 'will', 'stay', 'up', 'all', 'night', "I've", 'been', "goin'", 'hard', 'since', 'last', 'night', 'And', "I'mma", 'go', 'harder', 'tonight', 'Wish', 'you', 'could', 'see', 'you', 'through', 'my', 'eyes', 'Ooh', "I'm", 'telling', 'you', 'this', "ain't", 'the', 'same', 'And', 'I', 'know', "he's", 'still', 'in', 'your', 'brain', "I'm", "'bout", 'to', 'burn', 'that', 'shit', 'into', 'flames', 'Once', "I'm", 'in', 'you', 'Baby', 'Forget', 'what', 'you', 'know', 'Make', 'yourself', 'at', 'home', "'Cuz", 'baby', 'when', "I'm", 'finished', 'wit

In [7]:
# 計算原始歌詞的音節數量
lst_ori_syllables = []
for i in range(len(lst_ori_tokenize)):
    count_ori = syllable_count(lst_ori_tokenize[i])
    lst_ori_syllables.append(count_ori)
print(len(lst_ori_syllables))
print(lst_ori_syllables)

277
[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 2, 1, 2]


In [8]:
# 比對兩個音節個數陣列(gen長度>ori長度)
len_g = len(lst_gen_syllables)
len_o = len(lst_ori_syllables)
if len_g > len_o:
    lst_gen_syllables2 = lst_gen_syllables[:len_g-(len_g-len_o)]
# print('lst_gen_syllables2:',len(lst_gen_syllables2))
# print(lst_gen_syllables2)
correct = 0
wrong = 0
tmp_w = []
tmp_wo = []
tmp_s = []
tmp_so = []
for j in range(len(lst_gen_syllables2)):
    if lst_gen_syllables[j] == lst_ori_syllables[j]:
        tmp_w.append(lst_gen_tokenize[j])
        tmp_wo.append('')
        tmp_s.append(lst_gen_syllables[j])
        tmp_so.append('')
        correct += 1
    else:
        tmp_w.append('')
        tmp_wo.append(lst_gen_tokenize[j]) # 生成的詞語對不上原始歌詞的音節
        tmp_s.append('')
        tmp_so.append(lst_ori_syllables[j]) # 正確的音節數
        wrong += 1
print(correct)
print(wrong)
#print('tmp_w:',len(tmp_w)) # 有對齊的生成詞語
#print(tmp_w)
print('tmp_wo:',len(tmp_wo)) # 沒對齊的生成詞語
print(tmp_wo)
#print('tmp_s:',len(tmp_s)) # 有對齊的生成詞語音節數量
#print(tmp_s)
print('tmp_so:',len(tmp_so)) # 沒對齊的(取原始歌詞詞語應有的音節數量)
print(tmp_so)

213
64
tmp_wo: 277
['', '', '', '', '', '', '', 'lived', '', '', '', 'used', '', '', '', '', 'woods', '', '', "man's", 'religious', '', '', '', '', 'a', '', 'No', '', '', '', 'a', '', '', '', 'said', '', '', '', '', 'my', '', '', '', 'And', '', '', '', 'find', 'my', '', '', '', '', '', '', '', 'the', '', '', '', '', '', '', '', 'understand', '', '', '', '', '', '', 'start', 'living', 'for', '', '', '', '', '', '', '', '', '', '', 'living', 'for', 'today', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'No', 'I', '', '', '', '', 'Sometimes', 'I', '', '', '', '', 'wife', '', '', '', 'how', '', '', 'to', '', '', '', '', "doesn't", '', 'to', '', 'But', '', 'somehow', '', '', '', 'what', '', '', '', '', '', '', '', '', '', '', '', 'ready', '', 'begin', '', 'how', '', '', '', '', 'down', '', 'ready', 'ready', 'ready', '', '', 'can', '', '', '', '', '', '', 'I', '', '', 'understand', 'Then', '', '', '', '', '', '', 'living', '', '', '', '', '', '', 'to', '', 'down', '

In [78]:
# 測試用：搜尋詞語的同義詞(使用NLTK)
#nltk.download('wordnet')
num = 20
synonyms = []
word = tmp_wo[num]
try:
    syns = wordnet.synsets(word)[0].name()
except:
    word = preprocessText2(word) # 對詞語預處理
    syns = wordnet.synsets(word)[0].name()
print('original word:',word)
for syn in wordnet.synsets(word): # pos可指定詞性,pos='a'
    for l in syn.lemmas():
        synonyms.append(l.name())
synonyms2 = list(set(synonyms))
print('synonyms list:',synonyms2) # 用set()把重複的去除掉

s = tmp_so[num] # 正確的音節數量
print('correct syllable count:',s)
match_word = []
for n in range(len(synonyms2)):
    if syllable_count(synonyms2[n]) == s: # 列出符合音節數的同義詞
        print(synonyms2[n],syllable_count(synonyms2[n]))
        match_word.append(synonyms2[n])
print(match_word)
if len(match_word) == 0:
    crawl = crawl_synonyms(tmp_wo[num])
    print(crawl[0])

original word: religious
synonyms list: ['spiritual', 'religious']
correct syllable count: 1
[]
clerical


In [80]:
# 搜尋詞語的同義詞(使用NLTK)，並更新list(lst_rewrite)
lst_rewrite = []
for num in range(len(tmp_wo)): # len(tmp_wo)
    synonyms = []
    word = tmp_wo[num]
    if word != '':
        try:
            syns = wordnet.synsets(word)[0].name()
        except:
            word = preprocessText2(word) # 對詞語預處理
            syns = wordnet.synsets(word)[0].name()
        print('original word:',word)
        for syn in wordnet.synsets(word): # pos可指定詞性,pos='a'
            for l in syn.lemmas():
                synonyms.append(l.name())
        synonyms2 = list(set(synonyms))
        print('synonyms list:',synonyms2) # 用set()把重複的去除掉

        s = tmp_so[num] # 正確的音節數量
        print('correct syllable count:',s)
        match_word = []
        for n in range(len(synonyms2)):
            if syllable_count(synonyms2[n]) == s: # 列出符合音節數的同義詞
                #print(synonyms2[n],syllable_count(synonyms2[n]))
                match_word.append(synonyms2[n])
        if len(match_word) != 0: # 用wordnet方法找的到同義詞(match_word陣列長度不為0)
            print('match word:',match_word[0])
            lst_rewrite.append(match_word[0])
        else: # 方法一(wordnet方法)找不到同義詞，採用第二種方法抓取同義詞(def crawl_synonyms)
            crawl = crawl_synonyms(tmp_wo[num])
            print('match word:',crawl[0])
            lst_rewrite.append(crawl[0])
        print('-'*70)
    else:
        lst_rewrite.append('')
print(len(lst_rewrite))   
print(lst_rewrite)

original word: lived
synonyms list: ['live', 'last', 'endure', 'subsist', 'dwell', 'survive', 'exist', 'know', 'inhabit', 'hold_up', 'hold_out', 'go', 'be', 'experience', 'populate', 'live_on']
correct syllable count: 1
match word: live
----------------------------------------------------------------------
original word: used
synonyms list: ['victimized', 'ill-used', 'put-upon', 'employ', 'exploited', 'apply', 'use', 'habituate', 'secondhand', 'used', 'utilize', 'expend', 'utilise', 'practice', 'victimised']
correct syllable count: 1
match word: use
----------------------------------------------------------------------
original word: woods
synonyms list: ['Natalie_Wood', 'woodwind_instrument', 'Ellen_Price_Wood', 'woodwind', 'woods', 'Sir_Henry_Joseph_Wood', 'Grant_Wood', 'wood', 'forest', 'Wood', 'Sir_Henry_Wood', 'Mrs._Henry_Wood']
correct syllable count: 3
match word: Mrs._Henry_Wood
----------------------------------------------------------------------
original word: mans
synonyms 

IndexError: list index out of range

# Additional

In [46]:
# 對歌詞的文本預處理
def preprocessText(text, remove_stops=False):
    
    # 移除中括號'[]'的內容
    text = re.sub(pattern="\[.+?\]( )?", repl='', string=text)

    # Change "walkin'" to "walking"
    text = re.sub(pattern="n\\\' ", repl='ng ', string=text)

    # 移除小括號'()'，如：x4 → (x4)
    text = re.sub(pattern="(\()?x\d+(\))?", repl=' ', string=text)

    # 修正單引號
    text = re.sub(pattern="\\x91", repl="'", string=text) # \x保留字符
    text = re.sub(pattern="\\x92", repl="'", string=text)
    text = re.sub(pattern="<u\+0092>", repl="'", string=text)
    
    # 傳化成小寫
    text = text.lower()
    
    # 處理特殊字詞用法
    text = re.sub(pattern="'til", repl="til", string=text)
    text = re.sub(pattern="'til", repl="til", string=text)
    text = re.sub(pattern="gon'", repl="gon", string=text)

    # 處理開頭的換行符號'\n'
    text = re.sub(pattern='^\n', repl='', string=text)

    # 移除「,」、「!」、「?」、「,」、「\n」
    text = ''.join([char.strip(",!?:") for char in text])
    text = text.replace('\n', ' ')

    # 處理英文縮寫(特定)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"won\’t", "will not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"can\’t", "can not", text)
    text = re.sub(r"let's", "let us", text)
    text = re.sub(r"let\’s", "let us", text)
    text = re.sub(r"ain't", "aint", text)
    text = re.sub(r"ain\’t", "aint", text)

    # 處理英文縮寫(通用)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"n\’t", " not", text)
    text = re.sub(r"\’re", " are", text)
    text = re.sub(r"\’s", " is", text)
    text = re.sub(r"\’d", " would", text)
    text = re.sub(r"\’ll", " will", text)
    text = re.sub(r"\’t", " not", text)
    text = re.sub(r"\’ve", " have", text)
    text = re.sub(r"\’m", " am", text)

    # 移除剩餘的標點符號
    punc = string.punctuation
    text = ''.join([char for char in text if char not in punc])

    # 移除停用詞
    if remove_stops:
        stops = stopwords.words('english')
        text = ' '.join([word for word in text.split(' ') if word not in stops])
    
    # 刪除雙空格和開始結尾空格
    text = re.sub(pattern='( ){2,}', repl=' ', string=text)
    text = text.strip()
    
    return(text)

In [47]:
preprocessText(gen_text)

'where the old men pray his children lived next door they used to live in the woods but this man is religious bone he did not have a prayer no he did not have a prayer he just said i have got to find my own way home and i have got to find my own way home if i can do the things he says that i just can not understand then i will just settle down and start living for today lord how i would love to settle down and start living for today lord how i would love to settle down but i do not have a prayer i do not have a prayer no i do not have a prayer sometimes i think of what a wife must be and how i hate to do things that he does not have to do but then somehow i think of what hope is and do i have to start oh oh i am ready to begin oh how i would love to settle down i am ready ready ready if i can do the things he says that i just can not understand then i will just settle down and start living for today lord how i would love to settle down and i do not have a prayer no i do not have a pray

In [48]:
# 對歌詞的文本預處理版本2
def preprocessText2(text, remove_stops=False):
    
    # 移除中括號'[]'的內容
    text = re.sub(pattern="\[.+?\]( )?", repl='', string=text)

    # Change "walkin'" to "walking"
    text = re.sub(pattern="n\\\' ", repl='ng ', string=text)

    # 移除小括號'()'，如：x4 → (x4)
    text = re.sub(pattern="(\()?x\d+(\))?", repl=' ', string=text)

    # 修正單引號
    text = re.sub(pattern="\\x91", repl="'", string=text) # \x保留字符
    text = re.sub(pattern="\\x92", repl="'", string=text)
    text = re.sub(pattern="<u\+0092>", repl="'", string=text)
    
    # 傳化成小寫
    text = text.lower()
    
    # 處理特殊字詞用法
    text = re.sub(pattern="'til", repl="til", string=text)
    text = re.sub(pattern="'til", repl="til", string=text)
    text = re.sub(pattern="gon'", repl="gon", string=text)

    # 處理開頭的換行符號'\n'
    text = re.sub(pattern='^\n', repl='', string=text)

    # 移除「,」、「!」、「?」、「,」、「\n」
    text = ''.join([char.strip(",!?:") for char in text])
    text = text.replace('\n', ' ')

    # 移除剩餘的標點符號
    punc = string.punctuation
    text = ''.join([char for char in text if char not in punc])

    # 移除停用詞
    if remove_stops:
        stops = stopwords.words('english')
        text = ' '.join([word for word in text.split(' ') if word not in stops])
    
    # 刪除雙空格和開始結尾空格
    text = re.sub(pattern='( ){2,}', repl=' ', string=text)
    text = text.strip()
    
    return(text)

In [49]:
preprocessText2("man's")

'mans'

In [87]:
# 隨機產生英文詞語
r = RandomWords()
# Return a single random word
#r.get_random_word()
# Return list of Random words
#r.get_random_words()
# Return Word of the day
r.word_of_the_day()

'{"word": "cognoscitive", "definations": [{"source": "century", "text": "Having the power of knowing; cognitive.", "note": null, "partOfSpeech": "adjective"}]}'

In [100]:
# 隨機產生英文詞語2
vowels = list('aeiou')
def gen_word(min, max):
    word = ''
    syllables = min + int(random.random() * (max - min))
    for i in range(0, syllables):
        word += gen_syllable()
    return word.capitalize()
def gen_syllable():
    ran = random.random()
    if ran < 0.333:
        return word_part('v') + word_part('c')
    if ran < 0.666:
        return word_part('c') + word_part('v')
    return word_part('c') + word_part('v') + word_part('c')
def word_part(type):
    if type is 'c':
        return random.sample([ch for ch in list(string.ascii_lowercase) if ch not in vowels], 1)[0]
    if type is 'v':
        return random.sample(vowels, 1)[0]
for i in range(0, 50):
    print(gen_word(2,4))

Opic
Xiqalpef
Ojpor
Cakpofom
Jotekwo
Xumuq
Osfupsar
Weqo
Vaamik
Irivvo
Cuspavey
Besna
Buvucki
Igak
Qahet
Agogpe
Avkooh
Xezibqe
Fahyud
Bijzuhig
Ditzu
Uwoxok
Xudle
Vaobeg
Weahgif
Uvsina
Kesof
Urhewniz
Cavegi
Nodzuluh
Uyxec
Hixeik
Ugnepeq
Buhay
Oqluju
Qufe
Islafxab
Utdoli
Ibvoip
Ogje
Muzurop
Ejuyfa
Wokim
Sugebbe
Ivyub
Ecufwis
Eyacur
Efjon
Modkikoh
Qayufit


In [110]:
# 將詞語依據音節數分類，產生s1 s2 s3 s4 s5 s6
nouns = [
    'people',
    'history',
    'way',
    'art',
    'world',
    'information',
    'map',
    'two',
    'family',
    'government',
    'health',
    'system',
    'computer',
    'meat',
    'year',
    'thanks',
    'music',
    'person',
    'reading',
    'method',
    'data',
    'food',
    'understanding',
    'theory',
    'law',
    'bird',
    'literature',
    'problem',
    'software',
    'control',
    'knowledge',
    'power',
    'ability',
    'economics',
    'love',
    'internet',
    'television',
    'science',
    'library',
    'nature',
    'fact',
    'product',
    'idea',
    'temperature',
    'investment',
    'area',
    'society',
    'activity',
    'story',
    'industry',
    'media',
    'thing',
    'oven',
    'community',
    'definition',
    'safety',
    'quality',
    'development',
    'language',
    'management',
    'player',
    'variety',
    'video',
    'week',
    'security',
    'country',
    'exam',
    'movie',
    'organization',
    'equipment',
    'physics',
    'analysis',
    'policy',
    'series',
    'thought',
    'basis',
    'boyfriend',
    'direction',
    'strategy',
    'technology',
    'army',
    'camera',
    'freedom',
    'paper',
    'environment',
    'child',
    'instance',
    'month',
    'truth',
    'marketing',
    'university',
    'writing',
    'article',
    'department',
    'difference',
    'goal',
    'news',
    'audience',
    'fishing',
    'growth',
    'income',
    'marriage',
    'user',
    'combination',
    'failure',
    'meaning',
    'medicine',
    'philosophy',
    'teacher',
    'communication',
    'night',
    'chemistry',
    'disease',
    'disk',
    'energy',
    'nation',
    'road',
    'role',
    'soup',
    'advertising',
    'location',
    'success',
    'addition',
    'apartment',
    'education',
    'math',
    'moment',
    'painting',
    'politics',
    'attention',
    'decision',
    'event',
    'property',
    'shopping',
    'student',
    'wood',
    'competition',
    'distribution',
    'entertainment',
    'office',
    'population',
    'president',
    'unit',
    'category',
    'cigarette',
    'context',
    'introduction',
    'opportunity',
    'performance',
    'driver',
    'flight',
    'length',
    'magazine',
    'newspaper',
    'relationship',
    'teaching',
    'cell',
    'dealer',
    'debate',
    'finding',
    'lake',
    'member',
    'message',
    'phone',
    'scene',
    'appearance',
    'association',
    'concept',
    'customer',
    'death',
    'discussion',
    'housing',
    'inflation',
    'insurance',
    'mood',
    'woman',
    'advice',
    'blood',
    'effort',
    'expression',
    'importance',
    'opinion',
    'payment',
    'reality',
    'responsibility',
    'situation',
    'skill',
    'statement',
    'wealth',
    'application',
    'city',
    'county',
    'depth',
    'estate',
    'foundation',
    'grandmother',
    'heart',
    'perspective',
    'photo',
    'recipe',
    'studio',
    'topic',
    'collection',
    'depression',
    'imagination',
    'passion',
    'percentage',
    'resource',
    'setting',
    'ad',
    'agency',
    'college',
    'connection',
    'criticism',
    'debt',
    'description',
    'memory',
    'patience',
    'secretary',
    'solution',
    'administration',
    'aspect',
    'attitude',
    'director',
    'personality',
    'psychology',
    'recommendation',
    'response',
    'selection',
    'storage',
    'version',
    'alcohol',
    'argument',
    'complaint',
    'contract',
    'emphasis',
    'highway',
    'loss',
    'membership',
    'possession',
    'preparation',
    'steak',
    'union',
    'agreement',
    'cancer',
    'currency',
    'employment',
    'engineering',
    'entry',
    'interaction',
    'limit',
    'mixture',
    'preference',
    'region',
    'republic',
    'seat',
    'tradition',
    'virus',
    'actor',
    'classroom',
    'delivery',
    'device',
    'difficulty',
    'drama',
    'election',
    'engine',
    'football',
    'guidance',
    'hotel',
    'match',
    'owner',
    'priority',
    'protection',
    'suggestion',
    'tension',
    'variation',
    'anxiety',
    'atmosphere',
    'awareness',
    'bread',
    'climate',
    'comparison',
    'confusion',
    'construction',
    'elevator',
    'emotion',
    'employee',
    'employer',
    'guest',
    'height',
    'leadership',
    'mall',
    'manager',
    'operation',
    'recording',
    'respect',
    'sample',
    'transportation',
    'boring',
    'charity',
    'cousin',
    'disaster',
    'editor',
    'efficiency',
    'excitement',
    'extent',
    'feedback',
    'guitar',
    'homework',
    'leader',
    'mom',
    'outcome',
    'permission',
    'presentation',
    'promotion',
    'reflection',
    'refrigerator',
    'resolution',
    'revenue',
    'session',
    'singer',
    'tennis',
    'basket',
    'bonus',
    'cabinet',
    'childhood',
    'church',
    'clothes',
    'coffee',
    'dinner',
    'drawing',
    'hair',
    'hearing',
    'initiative',
    'judgment',
    'lab',
    'measurement',
    'mode',
    'mud',
    'orange',
    'poetry',
    'police',
    'possibility',
    'procedure',
    'queen',
    'ratio',
    'relation',
    'restaurant',
    'satisfaction',
    'sector',
    'signature',
    'significance',
    'song',
    'tooth',
    'town',
    'vehicle',
    'volume',
    'wife',
    'accident',
    'airport',
    'appointment',
    'arrival',
    'assumption',
    'baseball',
    'chapter',
    'committee',
    'conversation',
    'database',
    'enthusiasm',
    'error',
    'explanation',
    'farmer',
    'gate',
    'girl',
    'hall',
    'historian',
    'hospital',
    'injury',
    'instruction',
    'maintenance',
    'manufacturer',
    'meal',
    'perception',
    'pie',
    'poem',
    'presence',
    'proposal',
    'reception',
    'replacement',
    'revolution',
    'river',
    'son',
    'speech',
    'tea',
    'village',
    'warning',
    'winner',
    'worker',
    'writer',
    'assistance',
    'breath',
    'buyer',
    'chest',
    'chocolate',
    'conclusion',
    'contribution',
    'cookie',
    'courage',
    'dad',
    'desk',
    'drawer',
    'establishment',
    'examination',
    'garbage',
    'grocery',
    'honey',
    'impression',
    'improvement',
    'independence',
    'insect',
    'inspection',
    'inspector',
    'king',
    'ladder',
    'menu',
    'penalty',
    'piano',
    'potato',
    'profession',
    'professor',
    'quantity',
    'reaction',
    'requirement',
    'salad',
    'sister',
    'supermarket',
    'tongue',
    'weakness',
    'wedding',
    'affair',
    'ambition',
    'analyst',
    'apple',
    'assignment',
    'assistant',
    'bathroom',
    'bedroom',
    'beer',
    'birthday',
    'celebration',
    'championship',
    'cheek',
    'client',
    'consequence',
    'departure',
    'diamond',
    'dirt',
    'ear',
    'fortune',
    'friendship',
    'funeral',
    'gene',
    'girlfriend',
    'hat',
    'indication',
    'intention',
    'lady',
    'midnight',
    'negotiation',
    'obligation',
    'passenger',
    'pizza',
    'platform',
    'poet',
    'pollution',
    'recognition',
    'reputation',
    'shirt',
    'sir',
    'speaker',
    'stranger',
    'surgery',
    'sympathy',
    'tale',
    'throat',
    'trainer',
    'uncle',
    'youth',
    'time',
    'work',
    'film',
    'water',
    'money',
    'example',
    'while',
    'business',
    'study',
    'game',
    'life',
    'form',
    'air',
    'day',
    'place',
    'number',
    'part',
    'field',
    'fish',
    'back',
    'process',
    'heat',
    'hand',
    'experience',
    'job',
    'book',
    'end',
    'point',
    'type',
    'home',
    'economy',
    'value',
    'body',
    'market',
    'guide',
    'interest',
    'state',
    'radio',
    'course',
    'company',
    'price',
    'size',
    'card',
    'list',
    'mind',
    'trade',
    'line',
    'care',
    'group',
    'risk',
    'word',
    'fat',
    'force',
    'key',
    'light',
    'training',
    'name',
    'school',
    'top',
    'amount',
    'level',
    'order',
    'practice',
    'research',
    'sense',
    'service',
    'piece',
    'web',
    'boss',
    'sport',
    'fun',
    'house',
    'page',
    'term',
    'test',
    'answer',
    'sound',
    'focus',
    'matter',
    'kind',
    'soil',
    'board',
    'oil',
    'picture',
    'access',
    'garden',
    'range',
    'rate',
    'reason',
    'future',
    'site',
    'demand',
    'exercise',
    'image',
    'case',
    'cause',
    'coast',
    'action',
    'age',
    'bad',
    'boat',
    'record',
    'result',
    'section',
    'building',
    'mouse',
    'cash',
    'class',
    'nothing',
    'period',
    'plan',
    'store',
    'tax',
    'side',
    'subject',
    'space',
    'rule',
    'stock',
    'weather',
    'chance',
    'figure',
    'man',
    'model',
    'source',
    'beginning',
    'earth',
    'program',
    'chicken',
    'design',
    'feature',
    'head',
    'material',
    'purpose',
    'question',
    'rock',
    'salt',
    'act',
    'birth',
    'car',
    'dog',
    'object',
    'scale',
    'sun',
    'note',
    'profit',
    'rent',
    'speed',
    'style',
    'war',
    'bank',
    'craft',
    'half',
    'inside',
    'outside',
    'standard',
    'bus',
    'exchange',
    'eye',
    'fire',
    'position',
    'pressure',
    'stress',
    'advantage',
    'benefit',
    'box',
    'frame',
    'issue',
    'step',
    'cycle',
    'face',
    'item',
    'metal',
    'paint',
    'review',
    'room',
    'screen',
    'structure',
    'view',
    'account',
    'ball',
    'discipline',
    'medium',
    'share',
    'balance',
    'bit',
    'black',
    'bottom',
    'choice',
    'gift',
    'impact',
    'machine',
    'shape',
    'tool',
    'wind',
    'address',
    'average',
    'career',
    'culture',
    'morning',
    'pot',
    'sign',
    'table',
    'task',
    'condition',
    'contact',
    'credit',
    'egg',
    'hope',
    'ice',
    'network',
    'north',
    'square',
    'attempt',
    'date',
    'effect',
    'link',
    'post',
    'star',
    'voice',
    'capital',
    'challenge',
    'friend',
    'self',
    'shot',
    'brush',
    'couple',
    'exit',
    'front',
    'function',
    'lack',
    'living',
    'plant',
    'plastic',
    'spot',
    'summer',
    'taste',
    'theme',
    'track',
    'wing',
    'brain',
    'button',
    'click',
    'desire',
    'foot',
    'gas',
    'influence',
    'notice',
    'rain',
    'wall',
    'base',
    'damage',
    'distance',
    'feeling',
    'pair',
    'savings',
    'staff',
    'sugar',
    'target',
    'text',
    'animal',
    'author',
    'budget',
    'discount',
    'file',
    'ground',
    'lesson',
    'minute',
    'officer',
    'phase',
    'reference',
    'register',
    'sky',
    'stage',
    'stick',
    'title',
    'trouble',
    'bowl',
    'bridge',
    'campaign',
    'character',
    'club',
    'edge',
    'evidence',
    'fan',
    'letter',
    'lock',
    'maximum',
    'novel',
    'option',
    'pack',
    'park',
    'plenty',
    'quarter',
    'skin',
    'sort',
    'weight',
    'baby',
    'background',
    'carry',
    'dish',
    'factor',
    'fruit',
    'glass',
    'joint',
    'master',
    'muscle',
    'red',
    'strength',
    'traffic',
    'trip',
    'vegetable',
    'appeal',
    'chart',
    'gear',
    'ideal',
    'kitchen',
    'land',
    'log',
    'mother',
    'net',
    'party',
    'principle',
    'relative',
    'sale',
    'season',
    'signal',
    'spirit',
    'street',
    'tree',
    'wave',
    'belt',
    'bench',
    'commission',
    'copy',
    'drop',
    'minimum',
    'path',
    'progress',
    'project',
    'sea',
    'south',
    'status',
    'stuff',
    'ticket',
    'tour',
    'angle',
    'blue',
    'breakfast',
    'confidence',
    'daughter',
    'degree',
    'doctor',
    'dot',
    'dream',
    'duty',
    'essay',
    'father',
    'fee',
    'finance',
    'hour',
    'juice',
    'luck',
    'milk',
    'mouth',
    'peace',
    'pipe',
    'stable',
    'storm',
    'substance',
    'team',
    'trick',
    'afternoon',
    'bat',
    'beach',
    'blank',
    'catch',
    'chain',
    'consideration',
    'cream',
    'crew',
    'detail',
    'gold',
    'interview',
    'kid',
    'mark',
    'mission',
    'pain',
    'pleasure',
    'score',
    'screw',
    'sex',
    'shop',
    'shower',
    'suit',
    'tone',
    'window',
    'agent',
    'band',
    'bath',
    'block',
    'bone',
    'calendar',
    'candidate',
    'cap',
    'coat',
    'contest',
    'corner',
    'court',
    'cup',
    'district',
    'door',
    'east',
    'finger',
    'garage',
    'guarantee',
    'hole',
    'hook',
    'implement',
    'layer',
    'lecture',
    'lie',
    'manner',
    'meeting',
    'nose',
    'parking',
    'partner',
    'profile',
    'rice',
    'routine',
    'schedule',
    'swimming',
    'telephone',
    'tip',
    'winter',
    'airline',
    'bag',
    'battle',
    'bed',
    'bill',
    'bother',
    'cake',
    'code',
    'curve',
    'designer',
    'dimension',
    'dress',
    'ease',
    'emergency',
    'evening',
    'extension',
    'farm',
    'fight',
    'gap',
    'grade',
    'holiday',
    'horror',
    'horse',
    'host',
    'husband',
    'loan',
    'mistake',
    'mountain',
    'nail',
    'noise',
    'occasion',
    'package',
    'patient',
    'pause',
    'phrase',
    'proof',
    'race',
    'relief',
    'sand',
    'sentence',
    'shoulder',
    'smoke',
    'stomach',
    'string',
    'tourist',
    'towel',
    'vacation',
    'west',
    'wheel',
    'wine',
    'arm',
    'aside',
    'associate',
    'bet',
    'blow',
    'border',
    'branch',
    'breast',
    'brother',
    'buddy',
    'bunch',
    'chip',
    'coach',
    'cross',
    'document',
    'draft',
    'dust',
    'expert',
    'floor',
    'god',
    'golf',
    'habit',
    'iron',
    'judge',
    'knife',
    'landscape',
    'league',
    'mail',
    'mess',
    'native',
    'opening',
    'parent',
    'pattern',
    'pin',
    'pool',
    'pound',
    'request',
    'salary',
    'shame',
    'shelter',
    'shoe',
    'silver',
    'tackle',
    'tank',
    'trust',
    'assist',
    'bake',
    'bar',
    'bell',
    'bike',
    'blame',
    'boy',
    'brick',
    'chair',
    'closet',
    'clue',
    'collar',
    'comment',
    'conference',
    'devil',
    'diet',
    'fear',
    'fuel',
    'glove',
    'jacket',
    'lunch',
    'monitor',
    'mortgage',
    'nurse',
    'pace',
    'panic',
    'peak',
    'plane',
    'reward',
    'row',
    'sandwich',
    'shock',
    'spite',
    'spray',
    'surprise',
    'till',
    'transition',
    'weekend',
    'welcome',
    'yard',
    'alarm',
    'bend',
    'bicycle',
    'bite',
    'blind',
    'bottle',
    'cable',
    'candle',
    'clerk',
    'cloud',
    'concert',
    'counter',
    'flower',
    'grandfather',
    'harm',
    'knee',
    'lawyer',
    'leather',
    'load',
    'mirror',
    'neck',
    'pension',
    'plate',
    'purple',
    'ruin',
    'ship',
    'skirt',
    'slice',
    'snow',
    'specialist',
    'stroke',
    'switch',
    'trash',
    'tune',
    'zone',
    'anger',
    'award',
    'bid',
    'bitter',
    'boot',
    'bug',
    'camp',
    'candy',
    'carpet',
    'cat',
    'champion',
    'channel',
    'clock',
    'comfort',
    'cow',
    'crack',
    'engineer',
    'entrance',
    'fault',
    'grass',
    'guy',
    'hell',
    'highlight',
    'incident',
    'island',
    'joke',
    'jury',
    'leg',
    'lip',
    'mate',
    'motor',
    'nerve',
    'passage',
    'pen',
    'pride',
    'priest',
    'prize',
    'promise',
    'resident',
    'resort',
    'ring',
    'roof',
    'rope',
    'sail',
    'scheme',
    'script',
    'sock',
    'station',
    'toe',
    'tower',
    'truck',
    'witness',
    'a',
    'you',
    'it',
    'can',
    'will',
    'if',
    'one',
    'many',
    'most',
    'other',
    'use',
    'make',
    'good',
    'look',
    'help',
    'go',
    'great',
    'being',
    'few',
    'might',
    'still',
    'public',
    'read',
    'keep',
    'start',
    'give',
    'human',
    'local',
    'general',
    'she',
    'specific',
    'long',
    'play',
    'feel',
    'high',
    'tonight',
    'put',
    'common',
    'set',
    'change',
    'simple',
    'past',
    'big',
    'possible',
    'particular',
    'today',
    'major',
    'personal',
    'current',
    'national',
    'cut',
    'natural',
    'physical',
    'show',
    'try',
    'check',
    'second',
    'call',
    'move',
    'pay',
    'let',
    'increase',
    'single',
    'individual',
    'turn',
    'ask',
    'buy',
    'guard',
    'hold',
    'main',
    'offer',
    'potential',
    'professional',
    'international',
    'travel',
    'cook',
    'alternative',
    'following',
    'special',
    'working',
    'whole',
    'dance',
    'excuse',
    'cold',
    'commercial',
    'low',
    'purchase',
    'deal',
    'primary',
    'worth',
    'fall',
    'necessary',
    'positive',
    'produce',
    'search',
    'present',
    'spend',
    'talk',
    'creative',
    'tell',
    'cost',
    'drive',
    'green',
    'support',
    'glad',
    'remove',
    'return',
    'run',
    'complex',
    'due',
    'effective',
    'middle',
    'regular',
    'reserve',
    'independent',
    'leave',
    'original',
    'reach',
    'rest',
    'serve',
    'watch',
    'beautiful',
    'charge',
    'active',
    'break',
    'negative',
    'safe',
    'stay',
    'visit',
    'visual',
    'affect',
    'cover',
    'report',
    'rise',
    'walk',
    'white',
    'beyond',
    'junior',
    'pick',
    'unique',
    'anything',
    'classic',
    'final',
    'lift',
    'mix',
    'private',
    'stop',
    'teach',
    'western',
    'concern',
    'familiar',
    'fly',
    'official',
    'broad',
    'comfortable',
    'gain',
    'maybe',
    'rich',
    'save',
    'stand',
    'young',
    'heavy',
    'hello',
    'lead',
    'listen',
    'valuable',
    'worry',
    'handle',
    'leading',
    'meet',
    'release',
    'sell',
    'finish',
    'normal',
    'press',
    'ride',
    'secret',
    'spread',
    'spring',
    'tough',
    'wait',
    'brown',
    'deep',
    'display',
    'flow',
    'hit',
    'objective',
    'shoot',
    'touch',
    'cancel',
    'chemical',
    'cry',
    'dump',
    'extreme',
    'push',
    'conflict',
    'eat',
    'fill',
    'formal',
    'jump',
    'kick',
    'opposite',
    'pass',
    'pitch',
    'remote',
    'total',
    'treat',
    'vast',
    'abuse',
    'beat',
    'burn',
    'deposit',
    'print',
    'raise',
    'sleep',
    'somewhere',
    'advance',
    'anywhere',
    'consist',
    'dark',
    'double',
    'draw',
    'equal',
    'fix',
    'hire',
    'internal',
    'join',
    'kill',
    'sensitive',
    'tap',
    'win',
    'attack',
    'claim',
    'constant',
    'drag',
    'drink',
    'guess',
    'minor',
    'pull',
    'raw',
    'soft',
    'solid',
    'wear',
    'weird',
    'wonder',
    'annual',
    'count',
    'dead',
    'doubt',
    'feed',
    'forever',
    'impress',
    'nobody',
    'repeat',
    'round',
    'sing',
    'slide',
    'strip',
    'whereas',
    'wish',
    'combine',
    'command',
    'dig',
    'divide',
    'equivalent',
    'hang',
    'hunt',
    'initial',
    'march',
    'mention',
    'spiritual',
    'survey',
    'tie',
    'adult',
    'brief',
    'crazy',
    'escape',
    'gather',
    'hate',
    'prior',
    'repair',
    'rough',
    'sad',
    'scratch',
    'sick',
    'strike',
    'employ',
    'external',
    'hurt',
    'illegal',
    'laugh',
    'lay',
    'mobile',
    'nasty',
    'ordinary',
    'respond',
    'royal',
    'senior',
    'split',
    'strain',
    'struggle',
    'swim',
    'train',
    'upper',
    'wash',
    'yellow',
    'convert',
    'crash',
    'dependent',
    'fold',
    'funny',
    'grab',
    'hide',
    'miss',
    'permit',
    'quote',
    'recover',
    'resolve',
    'roll',
    'sink',
    'slip',
    'spare',
    'suspect',
    'sweet',
    'swing',
    'twist',
    'upstairs',
    'usual',
    'abroad',
    'brave',
    'calm',
    'concentrate',
    'estimate',
    'grand',
    'male',
    'mine',
    'prompt',
    'quiet',
    'refuse',
    'regret',
    'reveal',
    'rush',
    'shake',
    'shift',
    'shine',
    'steal',
    'suck',
    'surround',
    'anybody',
    'bear',
    'brilliant',
    'dare',
    'dear',
    'delay',
    'drunk',
    'female',
    'hurry',
    'inevitable',
    'invite',
    'kiss',
    'neat',
    'pop',
    'punch',
    'quit',
    'reply',
    'representative',
    'resist',
    'rip',
    'rub',
    'silly',
    'smile',
    'spell',
    'stretch',
    'stupid',
    'tear',
    'temporary',
    'tomorrow',
    'wake',
    'wrap',
    'yesterday']
s1 = []
s2 = []
s3 = []
s4 = []
s5 = []
s6 = []
s1_s6 = []
#print(" ".join([nouns[random.randrange(0, len(nouns))] for i in range(4)]))
for a in range(len(nouns)):
    count = syllable_count(nouns[a])
    if count == 1:
        s1.append(nouns[a])
    if count == 2:
        s2.append(nouns[a])
    if count == 3:
        s3.append(nouns[a])
    if count == 4:
        s4.append(nouns[a])
    if count == 5:
        s5.append(nouns[a])
    if count == 6:
        s6.append(nouns[a])
s1_s6.append(s1)
s1_s6.append(s2)
s1_s6.append(s3)
s1_s6.append(s4)
s1_s6.append(s5)
s1_s6.append(s6)
print(len(nouns),'(total) /',len(s1_s6),'(type) /',len(s1),'(s1) /',len(s2),'(s2) /',len(s3),'(s3) /',len(s4),'(s4) /',len(s5),'(s5) /',len(s6),'(s6)')
print('s1 =',s1)
print('s2 =',s2)
print('s3 =',s3)
print('s4 =',s4)
print('s5 =',s5)
print('s6 =',s6)

1524 (total) / 6 (type) / 709 (s1) / 493 (s2) / 227 (s3) / 79 (s4) / 15 (s5) / 1 (s6)
s1 = ['people', 'way', 'art', 'world', 'map', 'two', 'health', 'meat', 'year', 'thanks', 'food', 'law', 'bird', 'love', 'science', 'fact', 'thing', 'player', 'week', 'movie', 'thought', 'child', 'month', 'truth', 'goal', 'news', 'growth', 'night', 'disk', 'road', 'role', 'soup', 'math', 'wood', 'flight', 'length', 'cell', 'lake', 'phone', 'scene', 'death', 'mood', 'blood', 'skill', 'wealth', 'depth', 'heart', 'ad', 'debt', 'loss', 'steak', 'seat', 'match', 'bread', 'employee', 'guest', 'height', 'mall', 'sample', 'mom', 'church', 'coffee', 'hair', 'lab', 'mode', 'mud', 'queen', 'song', 'tooth', 'town', 'wife', 'gate', 'girl', 'hall', 'meal', 'pie', 'poem', 'son', 'speech', 'tea', 'breath', 'buyer', 'chest', 'cookie', 'dad', 'desk', 'king', 'tongue', 'apple', 'beer', 'cheek', 'client', 'dirt', 'ear', 'gene', 'hat', 'poet', 'shirt', 'sir', 'tale', 'throat', 'uncle', 'youth', 'time', 'work', 'film', 'whi

In [113]:
# 根據音節數，到特定的音節陣列中隨機挑選
c = 4
random.choice(s1_s6[c-1])

'elevator'

In [10]:
# 搜尋詞語的同義詞(使用NLTK)
nltk.download('wordnet')
synonyms = []
word = "good"
syns = wordnet.synsets(word)[0].name()
print('syns:',syns)
for syn in wordnet.synsets(word,pos='a'): # pos可指定詞性
    for l in syn.lemmas():
        synonyms.append(l.name())
synonyms2 = list(set(synonyms))
print(len(synonyms2))
print(synonyms2) # 用set()把重複的去除掉

[nltk_data] Downloading package wordnet to
[nltk_data]     D:\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
syns: good.n.01
32
['in_effect', 'near', 'just', 'expert', 'good', 'salutary', 'sound', 'beneficial', 'skilful', 'safe', 'unspoiled', 'proficient', 'effective', 'upright', 'honest', 'adept', 'well', 'practiced', 'dear', 'estimable', 'full', 'right', 'ripe', 'undecomposed', 'serious', 'skillful', 'secure', 'dependable', 'unspoilt', 'respectable', 'honorable', 'in_force']


In [11]:
# 找相似詞語的wordnet
lst_syns = []
for k in range(len(synonyms2)):
    a = synonyms2[k]
    b = wordnet.synsets(a)[0].name()
    #print(b)
    lst_syns.append(b)
print(lst_syns)

['effective.s.04', 'approach.v.01', 'just.a.01', 'expert.n.01', 'good.n.01', 'good.s.18', 'sound.n.01', 'beneficial.s.01', 'adept.s.01', 'safe.n.01', 'good.s.20', 'adept.s.01', 'effective.a.01', 'upright.n.01', 'honest.a.01', 'ace.n.03', 'well.n.01', 'practice.v.01', 'beloved.n.01', 'estimable.a.01', 'full_moon.n.01', 'right.n.01', 'ripe.a.01', 'good.s.20', 'serious.a.01', 'adept.s.01', 'procure.v.01', 'reliable.a.01', 'good.s.20', 'respectable.a.01', 'honest.a.01', 'effective.s.04']


In [12]:
# 相似詞語跟原始詞語的相似性計算
lst_sim = []
for w in range(len(lst_syns)):
    cb = wordnet.synset(syns)
    ib = wordnet.synset(lst_syns[w])
    sim = cb.wup_similarity(ib)
    lst_sim.append(sim)
print(lst_sim)

[None, None, None, 0.16666666666666666, 1.0, None, 0.46153846153846156, None, None, 0.11764705882352941, None, None, None, 0.11764705882352941, None, 0.15384615384615385, 0.14285714285714285, None, 0.15384615384615385, None, 0.5333333333333333, 0.25, None, None, None, None, None, None, None, None, None, None]


In [73]:
# 透過爬蟲方式找相似詞語
def synonyms(word):
    url = "https://www.thesaurus.com/browse/"
    url += word
    #print(url)
    r = requests.get(url)
    soup = bs(r.content, "html.parser")
    answer_list = soup.find("ul",'css-17d6qyx-WordGridLayoutBox et6tpn80')
    syns = answer_list.findChildren("a")
    #print(syns)
    lst = []
    for i in range(len(syns)):
        lst.append(syns[i].text)
    return print(lst)
synonyms_output = synonyms("lived")
synonyms_output

['alive', 'animate', 'breathing', 'living', 'aware', 'conscious', 'vital']


In [20]:
ps = PorterStemmer() 
w = tmp_wo[7] # 沒有對齊到的詞語
w = ps.stem(w)
print('original word:',w)
lst_syn = synonyms(w)
print('crawl list:',lst_syn)
s = tmp_so[7] # 正確的音節數量
print('correct syllable count:',s)
for n in range(len(lst_syn)):
    print(lst_syn[n],syllable_count(lst_syn[n]))
    if syllable_count(lst_syn[n]) == s:
        print(lst_syn[n])

original word: live
crawl list: ['alive', 'animate', 'breathing', 'living', 'aware', 'conscious', 'vital']
correct syllable count: 1
alive 2
animate 3
breathing 2
living 2
aware 2
conscious 2
vital 2


In [14]:
model = gensim.models.KeyedVectors.load_word2vec_format('model/GoogleNews-vectors-negative300.bin', binary=True)
model.wv['lived']

D:\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


array([ 1.05468750e-01,  4.19921875e-02, -9.94873047e-03,  7.22656250e-02,
        1.31835938e-01, -5.22460938e-02,  4.49218750e-02,  3.27148438e-02,
       -1.12915039e-02,  2.44140625e-01,  1.52343750e-01, -2.48046875e-01,
       -2.61718750e-01,  7.32421875e-02,  2.30712891e-02, -1.65039062e-01,
        3.75000000e-01,  3.33984375e-01, -3.54003906e-02,  4.88281250e-02,
       -2.63671875e-01,  1.95312500e-03, -2.06054688e-01, -1.67968750e-01,
        2.31445312e-01, -2.50000000e-01, -6.44531250e-02,  1.48437500e-01,
        4.95605469e-02, -1.66992188e-01, -1.15356445e-02, -9.57031250e-02,
       -1.02050781e-01,  6.59179688e-02, -1.73339844e-02, -8.17871094e-03,
        3.32031250e-01, -2.71484375e-01, -2.22656250e-01,  2.73437500e-01,
       -6.39648438e-02,  1.82617188e-01,  5.49316406e-03, -1.16210938e-01,
       -1.73568726e-04,  1.07421875e-01, -7.66601562e-02, -1.89453125e-01,
       -2.47070312e-01, -1.89453125e-01,  5.63964844e-02, -2.59765625e-01,
        3.19824219e-02, -

In [15]:
model.similar_by_word('lived', topn=10)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-bcf048eb238e>", line 1, in <module>
    model.similar_by_word('lived', topn=10)
  File "D:\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py", line 417, in similar_by_word
    return self.most_similar(positive=[word], topn=topn, restrict_vocab=restrict_vocab)
  File "D:\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py", line 348, in most_similar
    self.init_sims()
  File "D:\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py", line 1053, in init_sims
    self.vectors_norm = (self.vectors / sqrt((self.vectors ** 2).sum(-1))[..., newaxis]).astype(REAL)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 204

KeyboardInterrupt: 

In [ ]:
data = model.most_similar('lived')
print(data)

In [2]:
# https://github.com/danyalette/markov-poem
def makeId(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))
def getSourceCacheId(cacheIndex, source):
    cacheId = None
    for i in cacheIndex:
        if i[0] == source:
            cacheId = i[1]
    return cacheId
def removeDuplicateCachedModels(cacheIndex, source):
    for i in cacheIndex:
        if i[0] == source:
            cacheIndex.remove(i)
def writeJson(fileHandle, jsonString):
    # 解決unicode錯誤問題：https://zhidao.baidu.com/question/309967670532246284.html
    #return fileHandle.write(unicode(json.dumps(jsonString, ensure_ascii=False)))
    return fileHandle.write(json.dumps(jsonString, ensure_ascii=False))
def openFile(path, mode, isRelative = True):
    if isRelative:
        currentDir = os.path.dirname('test.txt')
        path = os.path.join(currentDir, path)
    return io.open(path,mode,encoding='utf-8')
def generatePoem(syllableCounts, sourceText, cache):

    result = []
    syllableCounts = syllableCounts.split(',')

    if not os.path.isfile(sourceText):
        throwUsageError()
    if not syllableCounts or [s for s in syllableCounts if not s.isdigit()]:
        throwUsageError()

    # get list of cached sources

    with openFile('data/sources.json', 'r') as f:
        text = f.read()
        if text != '':
            sourceModelIndex = json.loads(text)
        else:
            sourceModelIndex = []

    # get id of current source's cachec
    sourceCacheId = getSourceCacheId(sourceModelIndex, sourceText)

    # if ww to use cache, and current source is cached:
    if cache and sourceCacheId:
        with openFile('data/' + sourceCacheId + '.json', 'r') as f:
            data = json.load(f)
            text_model = markovify.Text.from_json(data)
    else:
        with openFile(sourceText, 'r', False) as f:
            text = f.read()
        text_model = markovify.Text(text)
        model_json = text_model.to_json()

        # save new markov model to cache
        sourceId = makeId()
        with openFile('data/sources.json', 'w') as indexFile:
            removeDuplicateCachedModels(sourceModelIndex, sourceText)
            sourceModelIndex.append((sourceText,sourceId))
            writeJson(indexFile, sourceModelIndex)
        with openFile('data/' + sourceId + '.json', 'w') as outFile:
            writeJson(outFile, model_json)

    for s in syllableCounts:
        errorMessage = '[Error generating line with length ' + s + ']'
        if int(s) > 2 and int(s) < 15:
            line = None
            i = 0
            while not line and i < 10:
                line = generateLine(int(s), text_model)
            if not line:
                line = errorMessage
        else:
            line = errorMessage
        result.append(line)
    return '\n'.join(result)
def throwUsageError():
    print('Usage: python markov_poem.py -f <source_text_filepath> -s <comma,separated,syllable,count> [-c (use cached markov model of source text)]')
    sys.exit()
def generateLine(syllables, text_model):
    for i in range(300):
        sentence = text_model.make_sentence()
        if (sentence):
            syls = getSyllables(sentence)
            if (syls == syllables):
                return sentence[:-1]
def main():
    sourceText = 'test.txt'
    syllableCounts = '5,7,5'
    cache = False
    print(generatePoem(syllableCounts, sourceText, cache))
main()

KeyboardInterrupt: 

In [131]:
# 計算音節數量
def count_syllables(word):
    return len(re.findall('(?!e$)[aeiouy]+', word, re.I) + re.findall('^[^aeiouy]*e$', word, re.I))
def syllable_count(word):
    word = word.lower()
    punc = string.punctuation
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    if word in punc: # 如果詞語是標點符號，音節數量則為0
        count = 0
    return count
def sylco(word) :
    word = word.lower()
    # exception_add are words that need extra syllables
    # exception_del are words that need less syllables
    exception_add = ['serious','crucial']
    exception_del = ['fortunately','unfortunately']
    co_one = ['cool','coach','coat','coal','count','coin','coarse','coup','coif','cook','coign','coiffe','coof','court']
    co_two = ['coapt','coed','coinci']
    pre_one = ['preach']
    syls = 0 #added syllable number
    disc = 0 #discarded syllable number
    #1) if letters < 3 : return 1
    if len(word) <= 3 :
        syls = 1
        return syls
    #2) if doesn't end with "ted" or "tes" or "ses" or "ied" or "ies", discard "es" and "ed" at the end.
    # if it has only 1 vowel or 1 set of consecutive vowels, discard. (like "speed", "fled" etc.)
    if word[-2:] == "es" or word[-2:] == "ed" :
        doubleAndtripple_1 = len(re.findall(r'[eaoui][eaoui]',word))
        if doubleAndtripple_1 > 1 or len(re.findall(r'[eaoui][^eaoui]',word)) > 1 :
            if word[-3:] == "ted" or word[-3:] == "tes" or word[-3:] == "ses" or word[-3:] == "ied" or word[-3:] == "ies" :
                pass
            else :
                disc+=1
    #3) discard trailing "e", except where ending is "le"  
    le_except = ['whole','mobile','pole','male','female','hale','pale','tale','sale','aisle','whale','while']
    if word[-1:] == "e" :
        if word[-2:] == "le" and word not in le_except :
            pass
        else :
            disc+=1
    #4) check if consecutive vowels exists, triplets or pairs, count them as one.
    doubleAndtripple = len(re.findall(r'[eaoui][eaoui]',word))
    tripple = len(re.findall(r'[eaoui][eaoui][eaoui]',word))
    disc+=doubleAndtripple + tripple
    #5) count remaining vowels in word.
    numVowels = len(re.findall(r'[eaoui]',word))
    #6) add one if starts with "mc"
    if word[:2] == "mc" :
        syls+=1
    #7) add one if ends with "y" but is not surrouned by vowel
    if word[-1:] == "y" and word[-2] not in "aeoui" :
        syls +=1
    #8) add one if "y" is surrounded by non-vowels and is not in the last word.
    for i,j in enumerate(word) :
        if j == "y" :
            if (i != 0) and (i != len(word)-1) :
                if word[i-1] not in "aeoui" and word[i+1] not in "aeoui" :
                    syls+=1
    #9) if starts with "tri-" or "bi-" and is followed by a vowel, add one.
    if word[:3] == "tri" and word[3] in "aeoui" :
        syls+=1
    if word[:2] == "bi" and word[2] in "aeoui" :
        syls+=1
    #10) if ends with "-ian", should be counted as two syllables, except for "-tian" and "-cian"
    if word[-3:] == "ian" : 
    #and (word[-4:] != "cian" or word[-4:] != "tian") :
        if word[-4:] == "cian" or word[-4:] == "tian" :
            pass
        else :
            syls+=1
    #11) if starts with "co-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.
    if word[:2] == "co" and word[2] in 'eaoui' :
        if word[:4] in co_two or word[:5] in co_two or word[:6] in co_two :
            syls+=1
        elif word[:4] in co_one or word[:5] in co_one or word[:6] in co_one :
            pass
        else :
            syls+=1
    #12) if starts with "pre-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.
    if word[:3] == "pre" and word[3] in 'eaoui' :
        if word[:6] in pre_one :
            pass
        else :
            syls+=1
    #13) check for "-n't" and cross match with dictionary to add syllable.
    negative = ["doesn't", "isn't", "shouldn't", "couldn't","wouldn't"]
    if word[-3:] == "n't" :
        if word in negative :
            syls+=1
        else :
            pass   
    #14) Handling the exceptional words.
    if word in exception_del :
        disc+=1
    if word in exception_add :
        syls+=1     
    # calculate the output
    return numVowels - disc + syls

In [132]:
word = ","
print(count_syllables(word))
print(syllable_count(word))
print(sylco(word))

0
0
1


In [43]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
data = 'Sadly, more downs than ups. The plot was pretty decent.'
for row in sent_tokenizer.tokenize(data):
    print(row)

Sadly, more downs than ups.
The plot was pretty decent.
